In [ ]:
%%writefile binary.cpp
#include <iostream>
#include<mpi/mpi.h>
#include<cstdlib>
using namespace std;

int flag=false;
void binarySearch(int *arr,int start, int end,int key,int rank)
{
	while(start<=end)
	{
		int mid=(start+end)/2;
		if(arr[mid]==key)
		{
			cout<<"Element is  Found by processor "<<rank<<" .\n";
			flag=true;
			return;
		}
		else if(arr[mid]<key)
		{
			start=mid+1;
		}
		else
		{
			end=mid-1;
		}
		//cout<<mid<<endl;
	}
}
int main(int argc, char **argv) {
	int n=6;
	int *arr=new int[n];

	for(int i=-3;i<4;i++)
	{
		arr[i]=i;
	}
	//cout<<"here";
	int key=4; //element to search
	MPI_Init(&argc, &argv);

	int rank,size;

	MPI_Comm_rank(MPI_COMM_WORLD, &rank);
	MPI_Comm_size(MPI_COMM_WORLD, &size);

	cout<<"Processor rank: "<<rank<<"\nSize : "<<size<<endl;// rank is process id to search
	int blocks=2;
	int blockSize=n/blocks;


	if(rank==0)
	{
		double start=MPI_Wtime();
		binarySearch(arr, rank*blockSize, (rank+1)*blockSize-1, key, rank);
		double end=MPI_Wtime();
		cout<<"Execution time of Processor "<<rank<<" is "<<(end-start)*1000<<endl;
	}
	else if(rank==1)
	{
		double start=MPI_Wtime();
		binarySearch(arr, rank*blockSize, (rank+1)*blockSize-1, key, rank);
		double end=MPI_Wtime();
		cout<<"Execution time of Processor "<<rank<<" is "<<(end-start)*1000<<endl;
	}
	else if(rank==2)
		{
			double start=MPI_Wtime();
			binarySearch(arr, rank*blockSize, (rank+1)*blockSize-1, key, rank);
			double end=MPI_Wtime();
			cout<<"Execution time of Processor "<<rank<<" is "<<(end-start)*1000<<endl;
		}
	else if(rank==3)
		{
			double start=MPI_Wtime();
			binarySearch(arr, rank*blockSize, (rank+1)*blockSize-1, key, rank);
			double end=MPI_Wtime();
			cout<<"Execution time of Processor "<<rank<<" is "<<(end-start)*1000<<endl;
		}
		

	MPI_Finalize();
	if(flag==false){
				cout<<"Element not found!";
		}

	return 0;
}

Overwriting binary.cpp


In [ ]:
!mpiCC binary.cpp
!mpirun --allow-run-as-root -np 4 ./a.out

Processor rank: 0
Size : 4
Element is  Found by processor 0 .
Execution time of Processor 0 is 0.004197
Processor rank: 3
Processor rank: 1
Size : 4
Execution time of Processor 1 is 0.000356
Processor rank: 2
Size : 4
Execution time of Processor 2 is 0.000487
Size : 4
Execution time of Processor 3 is 0.000612
Element not found!Element not found!Element not found!

In [ ]:
%%writefile bfs.cpp
#include<bits/stdc++.h>
#include "omp.h"

#define n_nodes 7

using namespace std;
list<int>q;
vector<int>weight(n_nodes,1000);
bool visited[n_nodes];

struct Comparator {
    // Compare 2 Edges objects using weight
    bool operator ()(const int &e1, const int &e2){
        return weight[e1]<weight[e2];
    }
};

//Display the list/priority queue - Debugging Function
void showlist(list <int> g) { 
    list <int> :: iterator it; 
    for(it = g.begin(); it != g.end(); ++it) 
        cout<<*it<<" "; 
    cout<<endl; 
}

//Display the vector/weights - Debugging Function
void showvec(vector <int> g) { 
    vector <int> :: iterator it; 
    for(it = g.begin(); it != g.end(); ++it) 
        cout<<*it<<" "; 
    cout<<endl; 
}

void bfs(int adj_matrix[n_nodes][n_nodes])
{
		if(q.empty())
			return;
    q.sort(Comparator());
		
		//pop first element and display it
		int cur_node = q.front();
    q.pop_front();
		printf("%d, ", cur_node);
	
		//For every element in the row of the adjacency matrix
		#pragma omp parallel for shared(visited,q,weight)
		for(int i=0; i<n_nodes; i++)
		{
			//If an unvisited Edge exists
			if(adj_matrix[cur_node][i]>0 && visited[i]==false)
			{

				//Replace the weight if it is larger
        if(weight[i] > adj_matrix[cur_node][i]){
            weight[i] = adj_matrix[cur_node][i];
        }

				//Push the destination of the smallest edge onto the queue
				q.push_back(i);
				visited[i]=true;
			}
		}
	
	//Call the function recursively
	bfs(adj_matrix);
}

int main(){
	
	//Get the maximum number of threads
  int th = omp_get_max_threads();
  cout<<"Number of Threads = "<<th<<endl;

	//Set the Adjacency Matrix
	int adj_matrix[n_nodes][n_nodes] = {
							{0	,11  ,15  ,0  ,0  ,0  ,0},
							{11	,0	,25	,25	,0	,0	,0},
							{15	,25	,0	,0	,40	,0	,0},
							{0	,25	,0	,0	,20	,0	,0},
							{0	,0	,40	,20	,0	,5	,0},
							{0	,0	,0	,0	,5	,0	,20},
							{0	,0	,0	,0	,0	,20	,0}
							};
	
	//Initialize visited boolean array
	for(int i=0; i<n_nodes; i++){
		visited[i] = false;
	}

	//Set the start node
	int start_node = 3;

	//Set up the queue, weight and visited
	q.push_back(start_node);
    weight[start_node] = 0;
	visited[start_node] = true;
	
	//Best first Search
	bfs(adj_matrix);

	return 0;
}

Overwriting bfs.cpp


In [ ]:
!g++ -fopenmp bfs.cpp
!./a.out

Number of Threads = 2
3, 4, 5, 6, 1, 0, 2, 